This file extracts the kinematics features from the GeoLife agents

Imports

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import math

First, we create a dictionary of agents who have labeled trips, since not all agents in the data include these labels

In [ ]:
# Glob all labels.txt files
labels = glob.glob('Geolife Trajectories 1.3/Data/*/labels.txt')

# Create dictionary of agent num -> trip dataframe
agent_trips = {}

for l in labels:

    # Get agent number from path
    agent_name = l[l.index('Data')+5:l.index('Data')+8]

    # Read labels.txt file
    with open(l) as text_file:
        trips = text_file.read().split('\n')
    text_file.close()

    # Split lines into lists
    trips = [x.split('\t') for x in trips]

    # Sometimes we get extra lines
    while len(trips[-1]) != 3:
        del trips[-1]

    # Transform to numpy
    trips = np.array(trips)

    # Transform again to dataframe
    trips = pd.DataFrame(data=trips[1:,:], columns=trips[0,:])

    # Alter columns appropriately
    trips['Agent_ID'] = int(agent_name)

    trips['Start_time'] = pd.to_datetime(trips['Start Time'], format='%Y/%m/%d %H:%M:%S')
    trips['End_time'] = pd.to_datetime(trips['End Time'], format='%Y/%m/%d %H:%M:%S')

    trips = trips.loc[:,['Agent_ID', 'Start_time','End_time', 'Transportation Mode']]

    # Add agent to dict
    agent_trips[int(agent_name)] = trips

This function reads in data from a single plt file, which usually represents the trajectory of a single day for a particular agent

In [ ]:
# Read the plt files for the agents
def extract_day(file_path):
    day = pd.read_csv(file_path, names=['latitude', 'longitude', 'zero', 'altitude', 'days', 'date', 'time'], skiprows=6)
    day['timestamp'] = pd.to_datetime(day['date'] + ' ' + day['time'])
    day = day.drop(columns=['zero', 'altitude', 'days', 'date', 'time'])

    return day

This function extracts the kinematic features for an agent, given:
(1) A dataframe of the trajectory for that agent
(2) A dataframe of the trips for that agent
(3) The name of the agent

In [ ]:
# Get kinematic profile, given a raw trajectory and trips
def get_kinematics_from_trips(ag_traj, ag_trips, ag_name):

    # Columns: See bottom
    trajectories = []

    # For distance/acceleration calculations
    some_const = math.cos(math.radians(ag_traj['latitude'][0]))
    R = 6371  # radius of the earth in km

    # Observe portions of trajectory that appear in a labeled trip
    for i in range(len(ag_trips)):
        trip = ag_traj[(ag_traj['timestamp'] <= ag_trips['End_time'].iloc[i]) & (ag_traj['timestamp'] >= ag_trips['Start_time'].iloc[i-1])]

        # Ignore if there is no movement or only one timestamp
        if len(trip) <= 1:
            continue

        trip = trip.to_numpy()

        movements = []

        # Cycle every timestamp of dataset
        j = 1
        while j < len(trip):
            
            # Note: Due to GPS errors, we sometimes have identical timestamps for the same agent, at different locations
            # To deal with this, we simply take the one which is closer to the previous timestamp

            # If there is no time between the current and previous timestamp, we must deal with this
            if trip[j][3] == 0:

                # One caveat: If the first two ticks of the trip are duplicate timestamps, we don't have a good solution. We simply take the last one
                if len(movements) == 0: # Equivalent should be if j == 1 but there could be more than two duplicates
                    j += 1
                    continue

                # Calculate distance from previous timestamp. First, find how many ticks backwards that is
                prev_tick = j - 2
                while trip[j][0] == trip[prev_tick][0]:
                    prev_tick -= 1

                # Add each move with these features: timestamp, speed, acceleration
                move = []
                move.append(trip[j][0])
                # Calculate distance
                d = 1000 * R * math.sqrt(
                    ((math.radians(trip[prev_tick][1]) - math.radians(trip[j][1])) * some_const) ** 2 + (
                            math.radians(trip[prev_tick][2]) - math.radians(trip[j][2])) ** 2)

                # Calculate speed using time since last timestamp
                d = d / trip[prev_tick + 1][3]

                move.append(d)
                if len(movements) == 1: # Here our check is if we have a single movement only, since that one must be the duplicate
                    move.append(d)  # Acceleration, since we start from stillness
                else:
                    move.append(d - movements[-1][1]) # Otherwise, calculate as difference in speed

                # Now, if this one has a shorter distance than the other tick at the same timestamp, replace it
                if move[1] < movements[-1][1]:
                    movements = movements[:-1]
                    movements.append(move)

                # Move to next timestamp
                j += 1
                continue

            # From here we have the typical case (no duplicate timestamp)
            
            # Add each move with these features: timestamp, speed, acceleration
            move = []
            move.append(trip[j][0])
            # Calculate distance
            d = 1000 * R * math.sqrt(
                ((math.radians(trip[j - 1][1]) - math.radians(trip[j][1])) * some_const) ** 2 + (
                        math.radians(trip[j - 1][2]) - math.radians(trip[j][2])) ** 2)

            # Calculate speed using time since last timestamp
            d = d / trip[j][3]

            move.append(d)
            if len(movements) == 0:
                move.append(d)  # Acceleration, since we start from stillness
            else:
                move.append(d - movements[-1][1]) # Otherwise, calculate as difference in speed
            movements.append(move)

            j += 1

        traj = []

        # From the list of movements with speeds and accelerations, we extract the kinematic features of the trip
        movements = np.array(movements)
        traj.append(ag_name)  # Agent_ID
        traj.append(trip[0][0])  # Start_time
        traj.append(trip[len(trip) - 1][0])  # End_time
        traj.append((trip[len(trip) - 1][0] - trip[0][0]).total_seconds())  # duration
        traj.append(max(movements[:, 1]))  # max_speed
        traj.append(min(movements[:, 1]))  # min_speed
        traj.append(max(movements[:, 2]))  # max_pos_acceleration
        traj.append(min(movements[:, 2]))  # max_neg_acceleration
        traj.append(np.mean(movements[:, 1]))  # avg_speed
        traj.append(np.mean(np.absolute(movements[:, 2])))  # avg_abs_acceleration
        traj.append(np.std(movements[:, 1]))  # std_dev_speed
        traj.append(np.std(movements[:, 2]))  # std_dev_acceleration
        traj.append(np.std(np.absolute(movements[:, 2])))  # std_dev_abs_acceleration

        traj.append(ag_trips['Transportation Mode'].iloc[i]) # modality

        # We record the kinematic features of each trip
        trajectories.append(traj)

    # Convert trajectories to a dataframe and return
    ret = pd.DataFrame(trajectories, columns =['Agent_ID',
                                                'Start_time', 
                                                'End_time', 
                                                'duration', 
                                                'max_speed', 
                                                'min_speed', 
                                                'max_pos_acceleration', 
                                                'max_neg_acceleration',
                                                'avg_speed', 
                                                'avg_abs_acceleration', 
                                                'std_dev_speed',
                                                'std_dev_acceleration',
                                                'std_dev_abs_acceleration',
                                                'modality'])

    return ret

Using the above functions, we cycle over the agents, extract their raw trajectories, and use these with their labeled trips to extract kinematic features

In [ ]:
agents = os.listdir('Geolife Trajectories 1.3/Data/')

kinematics = []

# Cycle agents
for agent in agents:

    #print('Agent=%d' % int(agent))

    # Ignore agent with no known trips
    if int(agent) not in agent_trips:
        continue

    #print('%d trips' % len(agent_trips[int(agent)]))

    # Extract raw trajectory
    agent_trajs = glob.glob('Geolife Trajectories 1.3/Data/'+agent+'/Trajectory/*.plt')
    agent_traj = pd.concat([extract_day(x) for x in agent_trajs], ignore_index=True)
    agent_traj = agent_traj.sort_values(by=['timestamp'])
    agent_traj = agent_traj.reset_index(drop=True)

    # Add number of seconds between each timestamp
    agent_traj['seconds_from_prev'] = (agent_traj['timestamp'] - agent_traj['timestamp'].shift()).apply(lambda x: x.total_seconds())

    # Change column order
    agent_traj = agent_traj.loc[:,['timestamp','longitude','latitude', 'seconds_from_prev']]

    kinematics.append(get_kinematics_from_trips(agent_traj, agent_trips[int(agent)], int(agent)))

kinematics = pd.concat(kinematics, ignore_index=True)
# Add agent name string
kinematics['agent_name'] = kinematics['Agent_ID'].apply(lambda x: 'agent=' + str(x).rjust(3, '0'))

We can observe some information about the dataframe we have created

In [ ]:
print(kinematics.columns)
print(kinematics.shape)
print(kinematics.head())

Now, we save the data to a file format/location of our choosing

In [ ]:
kinematics.to_feather('generated_data/kinematics.feather')

From here, we do some reductions. First, we remove trips which constitute kinematic outliers, by the following features

In [ ]:
features = ['duration', 'max_speed', 'min_speed', 'max_pos_acceleration', 'max_neg_acceleration', 'avg_speed',
            'avg_abs_acceleration', 'std_dev_speed', 'std_dev_acceleration', 'std_dev_abs_acceleration']

This function allows us to determine create a filter which excludes outliers

In [ ]:
# Remove outliers (by boxplot parameters)
def remove_outliers(feature, data):
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    filter = (data[feature] <= upper) & (data[feature] >= lower)

    return filter

We take only those trips which are not considered outliers

In [ ]:
filter = remove_outliers(features[0], kinematics)

for f in features[1:]:
    filter = (filter) & (remove_outliers(f, kinematics))

clean_df = kinematics[filter]
clean_df = clean_df.reset_index(drop=True)

Now, we can save these trips to a file format and location of our choosing

In [ ]:
clean_df.to_feather('generated_data/clean_kinematics.feather')

Lastly, we can remove agents which have less than 30 trips

In [ ]:
# Create a dictionary of agent num -> kinematics dataframe for that agent
clean_agents = clean_df.groupby(['Agent_ID'])
clean_agents = [clean_agents.get_group((x)) for x in clean_agents.groups]
clean_agents = [x.sort_values(by=['Start_time']) for x in clean_agents]
clean_agents = [x.reset_index(drop=True) for x in clean_agents]
clean_agents = {int(x['Agent_ID'].iloc[0]): x for x in clean_agents}

# Create a new dictionary which only inclues agents with at least 30 trips
reduced_agents = {k: v for k, v in clean_agents.items() if len(v) >= 30}

# Take the agents from this list and put them into a dataframe
reduced_df = clean_df[clean_df['Agent_ID'].isin(list(reduced_agents))]
reduced_df = reduced_df.reset_index(drop=True)

Finally, we save the data to a file format/location of our choosing. This is the final version of the data used in the paper

In [ ]:
reduced_df.to_feather('generated_data/reduced_kinematics.feather')

We can observe some information about the dataframe we have created

In [ ]:
print(reduced_df.columns)
print(reduced_df.shape)
print(reduced_df.head())